<a href="https://colab.research.google.com/github/Shakesdydaa/Autism-prediction-/blob/main/EDU_ROUTES_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [8]:
import numpy as np
import pandas as pd
import folium
from datetime import datetime, timedelta
import os
from math import radians, sin, cos, sqrt, atan2, degrees
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Configuration
SCHOOL_LOCATION = (-1.11767, 37.005043)
NUM_VEHICLES = 3
VEHICLE_CAPACITY = 18
AVERAGE_SPEED_KMPH = 35
MAX_STUDENTS = 50
SERVICE_TIME_PER_STUDENT = 3  # 3 minutes per student
MAX_ROUTE_DURATION_MIN = 120  # 2 hours maximum per route

# Natural geographic boundary angle instead of pure east/west
# Using an angled line that better fits Juja's geography
BOUNDARY_ANGLE = 165  # Degrees from north (adjusted from 180)

# Define buffer zone width for boundary refinement (in km)
BUFFER_ZONE_WIDTH = 0.5

# Modified Region Definition with improved boundary considerations
REGIONS = [
    {
        'name': 'Blue Region',
        'color': 'blue',
        'min_distance': 0,
        'max_distance': 1,  # 1 km radius
        'priority': 1,
        'sector': None  # No sector for inner region
    },
    {
        'name': 'Yellow West Region',
        'color': 'yellow',
        'min_distance': 1,
        'max_distance': 3,  # Extends to 3 km
        'priority': 2,
        'sector': 'west'
    },
    {
        'name': 'Red East Region',
        'color': 'red',
        'min_distance': 1,
        'max_distance': 3,  # Extends to 3 km
        'priority': 3,
        'sector': 'east'
    }
]

def haversine(lat1, lon1, lat2, lon2):
    """Calculate distance between coordinates in kilometers"""
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1-a))

def get_bearing(lat1, lon1, lat2, lon2):
    """Calculate initial bearing from point 1 to point 2"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    y = sin(dlon) * cos(lat2)
    x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    bearing = degrees(atan2(y, x))

    # Normalize to 0-360
    bearing = (bearing + 360) % 360
    return bearing

def is_west_sector(school_lat, school_lon, lat, lon):
    """
    Determine if a point is in the west sector using the adjusted boundary angle
    Accounts for the angled boundary rather than pure east/west division
    """
    bearing = get_bearing(school_lat, school_lon, lat, lon)

    # Instead of 180-360 degrees, use our custom boundary
    # West sector is now defined by BOUNDARY_ANGLE to (BOUNDARY_ANGLE + 180) % 360
    lower_bound = BOUNDARY_ANGLE
    upper_bound = (BOUNDARY_ANGLE + 180) % 360

    if lower_bound < upper_bound:
        return lower_bound <= bearing <= upper_bound
    else:
        # Handle case where the range crosses 0/360
        return bearing >= lower_bound or bearing <= upper_bound

def is_in_buffer_zone(school_lat, school_lon, lat, lon):
    """
    Determine if a point is in the buffer zone around the sector boundary
    Points in this zone can potentially be reassigned based on route efficiency
    """
    bearing = get_bearing(school_lat, school_lon, lat, lon)
    distance = haversine(school_lat, school_lon, lat, lon)

    # Skip inner blue region
    if distance <= REGIONS[0]['max_distance']:
        return False

    # Calculate how close the bearing is to the boundary
    angle_diff = min(
        abs(bearing - BOUNDARY_ANGLE) % 360,
        abs(bearing - (BOUNDARY_ANGLE + 180) % 360) % 360
    )

    # Convert angle difference to approximate distance at this radius
    # Using arc length formula: distance = radius * angle (in radians)
    arc_distance = distance * radians(angle_diff)

    # Return True if point is within buffer zone width
    return arc_distance <= BUFFER_ZONE_WIDTH

def categorize_students_by_region(locations):
    """
    Enhanced categorization that implements:
    1. Inner circular region (blue)
    2. West sector with adjusted boundary (yellow)
    3. East sector with adjusted boundary (red)
    4. Tracks buffer zone students for potential reassignment
    """
    school_lat, school_lon = SCHOOL_LOCATION
    student_regions = {region['name']: {'students': [], 'color': region['color']}
                       for region in REGIONS}

    buffer_zone_students = []

    for i, (lat, lon) in enumerate(locations[1:], 1):  # Skip school location
        # Calculate distance
        distance = haversine(school_lat, school_lon, lat, lon)

        # Check if in buffer zone for potential reassignment later
        if is_in_buffer_zone(school_lat, school_lon, lat, lon):
            buffer_zone_students.append(i)

        # Determine sector based on adjusted boundary angle
        is_west = is_west_sector(school_lat, school_lon, lat, lon)

        # Assign to appropriate region
        if distance < REGIONS[0]['max_distance']:
            # Inner blue region
            student_regions[REGIONS[0]['name']]['students'].append(i)
        else:
            # Outer regions - west or east based on adjusted boundary
            if is_west:
                student_regions[REGIONS[1]['name']]['students'].append(i)
            else:
                student_regions[REGIONS[2]['name']]['students'].append(i)

    return student_regions, buffer_zone_students

def generate_student_locations():
    """Generate realistic student locations in clusters"""
    if os.path.exists('juja_student_locations.csv'):
        return pd.read_csv('juja_student_locations.csv')

    np.random.seed(42)
    clusters = [
        {'center': (-1.105, 37.015), 'std': 0.008, 'weight': 0.35},
        {'center': (-1.130, 36.995), 'std': 0.007, 'weight': 0.30},
        {'center': (-1.115, 37.020), 'std': 0.006, 'weight': 0.20},
        {'center': SCHOOL_LOCATION, 'std': 0.003, 'weight': 0.15}
    ]

    locations = []
    for cluster in clusters:
        n = int(MAX_STUDENTS * cluster['weight'])
        lats = np.random.normal(cluster['center'][0], cluster['std'], n)
        lngs = np.random.normal(cluster['center'][1], cluster['std'], n)
        locations.extend(zip(lats, lngs))

    df = pd.DataFrame(locations[:MAX_STUDENTS], columns=['latitude', 'longitude'])
    df['student_id'] = [f'Student_{i+1}' for i in range(len(df))]
    df.to_csv('juja_student_locations.csv', index=False)
    return df

def create_distance_matrix(locations):
    """Create distance matrix in kilometers"""
    n = len(locations)
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                mat[i][j] = haversine(*locations[i], *locations[j])
    return mat

def calculate_route_impact(route, student_idx, distance_matrix):
    """
    Calculate the impact of adding a student to an existing route
    Returns the additional distance required to serve this student
    """
    if not route or len(route) < 2:
        # If route is empty or just has the school, impact is round trip distance
        return distance_matrix[0][student_idx] * 2

    min_impact = float('inf')

    # Find the best insertion point in the route
    for i in range(len(route) - 1):
        # Calculate distance if we insert student between i and i+1
        impact = (distance_matrix[route[i]][student_idx] +
                  distance_matrix[student_idx][route[i+1]] -
                  distance_matrix[route[i]][route[i+1]])

        if impact < min_impact:
            min_impact = impact

    return min_impact

def optimize_buffer_zone_assignments(student_regions, buffer_zone_students, routes, distance_matrix):
    """
    Reassign buffer zone students to optimize route efficiency
    This helps prevent routes from crossing into other sectors unnecessarily
    """
    if not buffer_zone_students:
        return student_regions

    # Create reverse lookup from student to region
    student_to_region = {}
    for region_name, region_data in student_regions.items():
        for student in region_data['students']:
            student_to_region[student] = region_name

    # For each buffer zone student, evaluate impact on each route
    for student in buffer_zone_students:
        current_region = student_to_region[student]

        # Skip blue region students entirely - they must stay in blue region
        if current_region == REGIONS[0]['name']:
            continue

        # Calculate impact on each route
        route_impacts = []
        for i, route in enumerate(routes):
            # Check if this route serves the student's region or adjacent regions
            route_region = get_predominant_region(route, student_to_region)

            # Skip routes that are predominantly blue if student is not blue
            if route_region == REGIONS[0]['name'] and current_region != REGIONS[0]['name']:
                continue

            impact = calculate_route_impact(route, student, distance_matrix)
            route_impacts.append((i, impact))

        if not route_impacts:
            continue

        # Find route with minimum impact
        best_route_idx, _ = min(route_impacts, key=lambda x: x[1])

        # Determine which region this route predominantly serves
        best_region = get_predominant_region(routes[best_route_idx], student_to_region)

        # If student would be better served by a different region, reassign
        if best_region != current_region:
            # Only allow reassignment if it follows constraints:
            # - Blue students stay in blue regions
            # - Yellow and red students can be reassigned between yellow and red
            if (current_region == REGIONS[0]['name'] and best_region != REGIONS[0]['name']):
                continue

            # Remove from current region
            student_regions[current_region]['students'].remove(student)
            # Add to best region
            student_regions[best_region]['students'].append(student)
            # Update reverse lookup
            student_to_region[student] = best_region

    return student_regions

def get_predominant_region(route, student_to_region):
    """Helper function to determine which region a route predominantly serves"""
    region_counts = {}
    for node in route:
        if node != 0 and node in student_to_region:  # Skip school
            region = student_to_region[node]
            region_counts[region] = region_counts.get(region, 0) + 1

    if not region_counts:  # Empty route or only school
        return None

    return max(region_counts.items(), key=lambda x: x[1])[0]

def optimize_regional_routes(distance_matrix, student_regions):
    """
    Optimize routes for each region with improved load balancing
    Ensures buses operate within their designated regions/sectors
    """
    routes = []
    current_routes_by_region = {region_name: [] for region_name in student_regions.keys()}

    # Sort regions by priority to handle closest students first
    sorted_regions = sorted(student_regions.items(), key=lambda x:
        next((r['priority'] for r in REGIONS if r['name'] == x[0]), float('inf'))
    )

    for region_name, region_data in sorted_regions:
        students = region_data['students']

        if not students:
            continue

        # For each region, create optimal routes using cluster-first, route-second approach
        remaining_students = students.copy()

        while remaining_students:
            route = [0]  # Start from school
            current = 0  # School index
            capacity = VEHICLE_CAPACITY

            while capacity > 0 and remaining_students:
                # Find closest remaining student to current position
                closest_idx = min(range(len(remaining_students)),
                                key=lambda i: distance_matrix[current][remaining_students[i]],
                                default=None)

                if closest_idx is None:
                    break

                next_student = remaining_students[closest_idx]
                route.append(next_student)
                current = next_student
                capacity -= 1

                # Remove the student from remaining list
                remaining_students.pop(closest_idx)

            route.append(0)  # Return to school
            routes.append(route)
            current_routes_by_region[region_name].append(route)

    return routes, current_routes_by_region

def smooth_routes(routes, distance_matrix, student_regions):
    """
    Post-processing to improve routes by identifying and fixing outliers
    Checks if any students would be better served by a different route
    """
    improved = True
    iteration = 0
    max_iterations = 5  # Limit iterations to prevent infinite loops

    # Create a mapping of which region each student belongs to
    student_to_region = {}
    for region_name, region_data in student_regions.items():
        for student in region_data['students']:
            student_to_region[student] = region_name

    # Create a mapping of which route each student belongs to
    student_to_route = {}
    for i, route in enumerate(routes):
        for student in route[1:-1]:  # Skip school at start and end
            student_to_route[student] = i

    while improved and iteration < max_iterations:
        improved = False
        iteration += 1

        # Update student to route mapping
        student_to_route = {}
        for i, route in enumerate(routes):
            for student in route[1:-1]:  # Skip school at start and end
                student_to_route[student] = i

        # For each route
        for route_idx, route in enumerate(routes):
            # Get the predominant region of this route
            route_region = get_predominant_region(route, student_to_region)

            # For each student in the route
            for i, student in enumerate(route[1:-1], 1):
                # Get student's assigned region
                student_region = student_to_region.get(student)

                # KEY CONSTRAINT: Blue students must only be in blue routes
                # If this student is in blue region but the route is not predominantly blue,
                # we need to find a better route for them
                if student_region == REGIONS[0]['name'] and route_region != REGIONS[0]['name']:
                    # Find a blue-predominant route to move this student to
                    for other_route_idx, other_route in enumerate(routes):
                        other_route_region = get_predominant_region(other_route, student_to_region)

                        # If this route serves blue region and has capacity
                        if (other_route_region == REGIONS[0]['name'] and
                            len(other_route) - 2 < VEHICLE_CAPACITY):  # -2 for school at start and end

                            # Find best insertion point
                            best_insert_pos = None
                            best_insert_cost = float('inf')

                            for j in range(1, len(other_route)):
                                insert_cost = (distance_matrix[other_route[j-1]][student] +
                                              distance_matrix[student][other_route[j]] -
                                              distance_matrix[other_route[j-1]][other_route[j]])

                                if insert_cost < best_insert_cost:
                                    best_insert_cost = insert_cost
                                    best_insert_pos = j

                            # Move student to the blue route
                            if best_insert_pos is not None:
                                routes[other_route_idx].insert(best_insert_pos, student)
                                routes[route_idx].remove(student)
                                improved = True
                                # No need to update student_to_region as it stays the same
                                break  # Found a suitable blue route

                    # Skip to next student if we've already processed this one
                    if improved:
                        break

                # For non-blue students or blue students already in blue routes, perform normal optimization
                elif student_region != REGIONS[0]['name']:
                    # Calculate current cost (distance)
                    current_cost = (distance_matrix[route[i-1]][student] +
                                   distance_matrix[student][route[i+1]] -
                                   distance_matrix[route[i-1]][route[i+1]])

                    # Try inserting this student into every other route
                    best_savings = 0
                    best_insert_route = None
                    best_insert_pos = None

                    for other_route_idx, other_route in enumerate(routes):
                        if other_route_idx == route_idx:
                            continue

                        # Check if this would violate capacity
                        if len(other_route) - 2 >= VEHICLE_CAPACITY:  # -2 for school at start and end
                            continue

                        # Get predominant region of other route
                        other_route_region = get_predominant_region(other_route, student_to_region)

                        # KEY CONSTRAINT: Non-blue students should not be moved to blue routes
                        if student_region != REGIONS[0]['name'] and other_route_region == REGIONS[0]['name']:
                            continue

                        # Find best insertion point
                        for j in range(1, len(other_route)):
                            # Cost of insertion into other route
                            insert_cost = (distance_matrix[other_route[j-1]][student] +
                                          distance_matrix[student][other_route[j]] -
                                          distance_matrix[other_route[j-1]][other_route[j]])

                            # If this improves overall distance
                            savings = current_cost - insert_cost
                            if savings > best_savings:
                                best_savings = savings
                                best_insert_route = other_route_idx
                                best_insert_pos = j

                    # If we found a better placement
                    if best_savings > 0.2:  # Small threshold to avoid tiny improvements
                        # Move student to better route
                        routes[best_insert_route].insert(best_insert_pos, student)
                        routes[route_idx].remove(student)
                        improved = True

                        # Find the predominant region of the destination route
                        new_route_region = get_predominant_region(routes[best_insert_route], student_to_region)

                        # Update region assignment if needed
                        if new_route_region != student_region and new_route_region is not None:
                            # Remove from current region
                            student_regions[student_region]['students'].remove(student)
                            # Add to new region
                            student_regions[new_route_region]['students'].append(student)
                            # Update student_to_region mapping
                            student_to_region[student] = new_route_region

                        # Don't continue checking current route as it's been modified
                        break

            # If we made improvements, restart the outer loop
            if improved:
                break

    return routes, student_regions

def calculate_arrival_times(routes, distance_matrix, is_pickup):
    """Calculate more accurate arrival times"""
    base_time = 480 if is_pickup else 1020  # 8:00 AM or 5:00 PM in minutes
    arrival_times = {0: base_time}  # School start/end time

    for route in routes:
        current_time = base_time
        for i in range(1, len(route)):
            prev_node = route[i-1]
            current_node = route[i]

            # Calculate travel time
            travel_time = distance_matrix[prev_node][current_node] / AVERAGE_SPEED_KMPH * 60

            # Add service time for pickups/dropoffs (except at school)
            if current_node != 0 and prev_node != 0:
                current_time += SERVICE_TIME_PER_STUDENT

            current_time += travel_time
            arrival_times[current_node] = current_time

    return arrival_times

def calculate_routes(locations, is_pickup):
    """Calculate optimized routes with improved sectoral boundaries and constraints"""
    distance_matrix = create_distance_matrix(locations)

    # Step 1: Categorize students by region with adjusted boundary
    student_regions, buffer_zone_students = categorize_students_by_region(locations)

    # Step 2: Create initial routes based on regions
    initial_routes, routes_by_region = optimize_regional_routes(distance_matrix, student_regions)

    # Step 3: Optimize buffer zone assignments, but prevent blue students from leaving blue routes
    student_regions = optimize_buffer_zone_assignments(
        student_regions, buffer_zone_students, initial_routes, distance_matrix)

    # Step 4: Re-optimize routes based on updated assignments
    routes, _ = optimize_regional_routes(distance_matrix, student_regions)

    # Step 5: Post-processing to smooth routes and enforce blue zone constraint
    routes, student_regions = smooth_routes(routes, distance_matrix, student_regions)

    # Step 6: Calculate arrival times
    arrival_times = calculate_arrival_times(routes, distance_matrix, is_pickup)

    return routes, arrival_times, student_regions

def visualize_regional_routes(locations, routes, arrival_times, is_pickup, student_regions):
    """Create interactive map visualization with improved boundary visualization"""
    m = folium.Map(location=SCHOOL_LOCATION, zoom_start=12)

    # School marker
    folium.Marker(
        SCHOOL_LOCATION,
        popup='<b>Juja Preparatory School</b>',
        icon=folium.Icon(color='green', icon='home')
    ).add_to(m)

    # Add inner circular region
    folium.Circle(
        SCHOOL_LOCATION,
        radius=REGIONS[0]['max_distance'] * 1000,  # Convert to meters
        color=REGIONS[0]['color'],
        fill=True,
        fill_opacity=0.1,
        popup=f"{REGIONS[0]['name']}: 0-{REGIONS[0]['max_distance']} km"
    ).add_to(m)

    # Add outer circular region boundary
    folium.Circle(
        SCHOOL_LOCATION,
        radius=REGIONS[1]['max_distance'] * 1000,  # Convert to meters
        color='gray',
        fill=False,
        weight=1,
        popup=f"Outer Boundary: {REGIONS[1]['max_distance']} km"
    ).add_to(m)

    # Add angled sector dividing lines instead of pure north-south
    # Calculate points 3km away at the boundary angles
    boundary_radians = radians(BOUNDARY_ANGLE)
    opposite_radians = radians((BOUNDARY_ANGLE + 180) % 360)

    # Distance to draw the boundary line (3 km to match outer boundary)
    line_distance = REGIONS[1]['max_distance']

    # Calculate end points of boundary lines
    bound1_lat = SCHOOL_LOCATION[0] + line_distance * cos(boundary_radians) / 111.32
    bound1_lon = SCHOOL_LOCATION[1] + line_distance * sin(boundary_radians) / (111.32 * cos(radians(SCHOOL_LOCATION[0])))

    bound2_lat = SCHOOL_LOCATION[0] + line_distance * cos(opposite_radians) / 111.32
    bound2_lon = SCHOOL_LOCATION[1] + line_distance * sin(opposite_radians) / (111.32 * cos(radians(SCHOOL_LOCATION[0])))

    # First boundary line
    folium.PolyLine(
        [SCHOOL_LOCATION, (bound1_lat, bound1_lon)],
        color='black',
        weight=2,
        opacity=0.7,
        dash_array='5,10',
        popup=f"Sector Boundary: {BOUNDARY_ANGLE}°"
    ).add_to(m)

    # Second boundary line
    folium.PolyLine(
        [SCHOOL_LOCATION, (bound2_lat, bound2_lon)],
        color='black',
        weight=2,
        opacity=0.7,
        dash_array='5,10',
        popup=f"Sector Boundary: {(BOUNDARY_ANGLE + 180) % 360}°"
    ).add_to(m)

    # Add buffer zone visualization
    buffer_points = []
    school_lat, school_lon = SCHOOL_LOCATION

    # Generate points to visualize buffer zone
    for angle in range(0, 360, 10):
        for dist in np.linspace(REGIONS[0]['max_distance'], REGIONS[1]['max_distance'], 5):
            angle_rad = radians(angle)
            lat = school_lat + dist * cos(angle_rad) / 111.32
            lon = school_lon + dist * sin(angle_rad) / (111.32 * cos(radians(school_lat)))

            if is_in_buffer_zone(school_lat, school_lon, lat, lon):
                buffer_points.append((lat, lon))

    # Add buffer zone points as small markers
    for point in buffer_points:
        folium.CircleMarker(
            point,
            radius=1,
            color='purple',
            fill=True,
            fill_opacity=0.5
        ).add_to(m)

    # Create a mapping of which region each student belongs to
    student_to_region = {}
    for region_name, region_data in student_regions.items():
        for student in region_data['students']:
            student_to_region[student] = region_name

    # Visualize routes with updated colors
    route_colors = {
        REGIONS[0]['name']: REGIONS[0]['color'],
        REGIONS[1]['name']: REGIONS[1]['color'],
        REGIONS[2]['name']: REGIONS[2]['color']
    }

    for i, route in enumerate(routes):
        # Determine route color based on predominant region
        route_region = get_predominant_region(route, student_to_region) or REGIONS[0]['name']
        route_color = route_colors.get(route_region, 'purple')

        # Calculate route locations
        path_locations = [locations[node] for node in route]

        # Draw route
        folium.PolyLine(
            path_locations,
            color=route_color,
            weight=4,
            popup=f"Route {i+1}: {len(route)-2} students | {route_region}"
        ).add_to(m)

        # Add student markers
        for node in route[1:-1]:
            # Determine student's region color
            student_color = 'green'  # Default
            if node in student_to_region:
                student_region = student_to_region[node]
                student_color = next((r['color'] for r in REGIONS if r['name'] == student_region), 'green')

            folium.CircleMarker(
                locations[node],
                radius=5,
                color=student_color,
                fill=True,
                popup=f"Student {node} - {student_to_region.get(node, 'Unknown')} Region"
            ).add_to(m)

    filename = f"constrained_sectoral_{'pickup' if is_pickup else 'dropoff'}_routes.html"
    m.save(filename)
    print(f"Map saved to {filename}")

def print_regional_schedule(routes, locations, arrival_times, is_pickup, student_regions):
    """Print detailed schedule with regional routing information"""
    print(f"\n{'MORNING PICKUP' if is_pickup else 'EVENING DROPOFF'} SCHEDULE (IMPROVED)")
    print("="*60)

    for i, route in enumerate(routes):
        print(f"\nRoute {i+1}:")

        # Determine the predominant region of this route
        region_counts = {region_name: 0 for region_name in student_regions.keys()}
        for node in route[1:-1]:
            for region_name, region_data in student_regions.items():
                if node in region_data['students']:
                    region_counts[region_name] += 1
                    break

        predominant_region = max(region_counts.items(), key=lambda x: x[1])[0] if region_counts else "Mixed"

        print(f"Region: {predominant_region}")
        print(f"Students: {len(route)-2}")

        print("\nRoute Details:")
        total_distance = 0
        for j in range(len(route)-1):
            dist = haversine(*locations[route[j]], *locations[route[j+1]])
            total_distance += dist

            if route[j] in arrival_times:
                start_time = datetime.strptime("00:00", "%H:%M") + timedelta(minutes=arrival_times[route[j]])
            else:
                start_time = datetime.strptime("00:00", "%H:%M")

            if route[j+1] in arrival_times:
                end_time = datetime.strptime("00:00", "%H:%M") + timedelta(minutes=arrival_times[route[j+1]])
            else:
                end_time = datetime.strptime("00:00", "%H:%M")

            if j == 0:
                print(f"  🏫 Depart School at {start_time:%H:%M}")
            elif j == len(route)-2:
                print(f"  🏫 {'Arrive at School' if is_pickup else 'Depart School'} at {end_time:%H:%M}")
            else:
                print(f"  🚏 Student {route[j]} {'Pickup' if is_pickup else 'Dropoff'} at {end_time:%H:%M}")

        print(f"\nTotal Route Distance: {total_distance:.2f} km")

        # Calculate route metrics
        route_duration = arrival_times[route[-1]] - arrival_times[route[0]]
        print(f"Route Duration: {route_duration:.1f} minutes")
        print(f"Students per Bus: {len(route)-2}")

def main():
    print("IMPROVED SECTORAL SCHOOL BUS ROUTING")
    print("="*50)
    print(f"Using adjusted boundary angle: {BOUNDARY_ANGLE}° from north")
    print(f"Buffer zone width: {BUFFER_ZONE_WIDTH} km")

    # Generate student locations
    df = generate_student_locations()
    locations = [SCHOOL_LOCATION] + list(zip(df['latitude'], df['longitude']))

    # Pickup Routes
    print("\nMorning Pickup Routes:")
    pickup_routes, pickup_times, pickup_regions = calculate_routes(locations, is_pickup=True)
    visualize_regional_routes(locations, pickup_routes, pickup_times, is_pickup=True, student_regions=pickup_regions)
    print_regional_schedule(pickup_routes, locations, pickup_times, is_pickup=True, student_regions=pickup_regions)

    # Dropoff Routes
    print("\nEvening Dropoff Routes:")
    dropoff_routes, dropoff_times, dropoff_regions = calculate_routes(locations, is_pickup=False)
    visualize_regional_routes(locations, dropoff_routes, dropoff_times, is_pickup=False, student_regions=dropoff_regions)
    print_regional_schedule(dropoff_routes, locations, dropoff_times, is_pickup=False, student_regions=dropoff_regions)

    # Print region statistics and impact of optimization
    print("\nREGION STATISTICS (AFTER OPTIMIZATION)")
    print("="*50)
    for region_name, region_data in pickup_regions.items():
        print(f"{region_name}: {len(region_data['students'])} students")

    # Calculate total distance for all routes
    total_distance = 0
    for route in pickup_routes:
        for j in range(len(route)-1):
            total_distance += haversine(*locations[route[j]], *locations[route[j+1]])

    print(f"\nTotal Distance (All Pickup Routes): {total_distance:.2f} km")
    print(f"Average Students Per Bus: {sum(len(r)-2 for r in pickup_routes)/len(pickup_routes):.1f}")
    print(f"Route Coherence Improved with Buffer Zone: {BUFFER_ZONE_WIDTH} km and Boundary Angle: {BOUNDARY_ANGLE}°")

if __name__ == '__main__':
    main()

IMPROVED SECTORAL SCHOOL BUS ROUTING
Using adjusted boundary angle: 165° from north
Buffer zone width: 0.5 km

Morning Pickup Routes:
Map saved to constrained_sectoral_pickup_routes.html

MORNING PICKUP SCHEDULE (IMPROVED)

Route 1:
Region: Blue Region
Students: 12

Route Details:
  🏫 Depart School at 08:22
  🚏 Student 49 Pickup at 08:03
  🚏 Student 48 Pickup at 08:06
  🚏 Student 46 Pickup at 08:10
  🚏 Student 45 Pickup at 08:13
  🚏 Student 17 Pickup at 08:17
  🚏 Student 33 Pickup at 08:22
  🚏 Student 44 Pickup at 08:25
  🚏 Student 47 Pickup at 08:29
  🚏 Student 24 Pickup at 08:34
  🚏 Student 43 Pickup at 08:37
  🚏 Student 38 Pickup at 08:40
  🏫 Arrive at School at 08:22

Total Route Distance: 5.27 km
Route Duration: 0.0 minutes
Students per Bus: 12

Route 2:
Region: Yellow West Region
Students: 14

Route Details:
  🏫 Depart School at 08:22
  🚏 Student 23 Pickup at 08:06
  🚏 Student 27 Pickup at 08:10
  🚏 Student 30 Pickup at 08:13
  🚏 Student 26 Pickup at 08:16
  🚏 Student 20 Pickup a